In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
data=[]
labels=[]
Parasitized=os.listdir("C:/Users/vivek/Downloads/cell_images/Parasitized/")
for a in Parasitized:
    try:
        image=cv2.imread("C:/Users/vivek/Downloads/cell_images/Parasitized/"+a)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")

Uninfected=os.listdir("C:/Users/vivek/Downloads/cell_images/Uninfected/")
for b in Uninfected:
    try:
        image=cv2.imread("C:/Users/vivek/Downloads/cell_images/Uninfected/"+b)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")

In [3]:
imgs=np.array(data)
labels=np.array(labels)

In [4]:
s=np.arange(imgs.shape[0])
np.random.shuffle(s)
imgs=imgs[s]
labels=labels[s]

In [5]:
num_classes=len(np.unique(labels))
len_data=len(imgs)

In [6]:
(x_train,x_test)=imgs[(int)(0.1*len_data):],imgs[:(int)(0.1*len_data)]
x_train = x_train.astype('float32')/255 # As we are working on image data we are normalizing data by divinding 255.
x_test = x_test.astype('float32')/255
train_len=len(x_train)
test_len=len(x_test)

In [7]:
(y_train,y_test)=labels[(int)(0.1*len_data):],labels[:(int)(0.1*len_data)]

In [8]:
num_classes=len(np.unique(labels))
Y_train=keras.utils.to_categorical(y_train,num_classes)
Y_test=keras.utils.to_categorical(y_test,num_classes)

In [9]:
VGG=VGG16(weights='imagenet', include_top=False, input_shape=(50, 50,3))

W0710 00:15:46.588416 21540 deprecation_wrapper.py:119] From C:\Users\vivek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0710 00:15:46.611354 21540 deprecation_wrapper.py:119] From C:\Users\vivek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 00:15:46.626324 21540 deprecation_wrapper.py:119] From C:\Users\vivek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0710 00:15:46.663250 21540 deprecation_wrapper.py:119] From C:\Users\vivek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0710 00:15:47.007363 21540 deprecation_wrapper.py:119] From C:\Users\vivek\Ana

In [10]:
model = Sequential()
for layer in VGG.layers:
    model.add(layer)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 12, 12, 256)       295168    
__________

In [12]:
model.layers.pop()

In [13]:
model.add(Flatten())
model.add(Dense(300, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))

W0710 00:15:52.340915 21540 deprecation.py:506] From C:\Users\vivek\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 50, 50, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 50, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 25, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 25, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 25, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 12, 12, 256)       295168    
__________

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

W0710 00:15:59.403174 21540 deprecation_wrapper.py:119] From C:\Users\vivek\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [16]:
model.fit(x_train,Y_train,batch_size=50,epochs=20,verbose=2)

W0710 00:16:00.596651 21540 deprecation.py:323] From C:\Users\vivek\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
 - 1116s - loss: 8.0282 - acc: 0.5013
Epoch 2/20
 - 1101s - loss: 8.0451 - acc: 0.5009
Epoch 3/20
 - 1136s - loss: 8.0451 - acc: 0.5009
Epoch 4/20
 - 1118s - loss: 8.0451 - acc: 0.5009
Epoch 5/20
 - 1131s - loss: 8.0451 - acc: 0.5009
Epoch 6/20


KeyboardInterrupt: 

In [25]:
accuracy = model.evaluate(x_test, Y_test, verbose=2)
print('Test_Accuracy:-', accuracy[1])

Test_Accuracy:- 0.9150635208711434
